In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Setup notebook
from pathlib import Path

# import necessary package
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.sequence import TimeseriesGenerator

import sklearn
from sklearn import preprocessing

# 畫圖表用
import matplotlib.pyplot as plt

In [ ]:
train_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv', index_col=0, parse_dates=['date'], infer_datetime_format=True)
test_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv', index_col=0, parse_dates=['date'], infer_datetime_format=True)
oil_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv', parse_dates=['date'], infer_datetime_format=True)
sub_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv')
holiday_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')
store_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
trans_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')

def check_data(df):
    print("-------------------------------TOP 10 RECORDS-----------------------------")
    print(df.head(10))
#     print("-------------------------------INFO--------------------------------------")
#     print(df.info())
#     print("-------------------------------Describe----------------------------------")
#     print(df.describe())
    print("-------------------------------Columns-----------------------------------")
    print(df.columns)
    print("-------------------------------Data Types--------------------------------")
    print(df.dtypes)
    print("----------------------------Missing Values-------------------------------")
    print(df.isnull().sum())
    print("----------------------------NULL values----------------------------------")
    print(df.isna().sum())
    print("--------------------------Shape Of Data---------------------------------")
    print(df.shape)
    print("============================================================================ \n")

In [ ]:
print("=================================train_df=================================")
check_data(train_df)
# print("=================================test_df=================================")
# check_data(test_df)
# print("=================================oil_df=================================")
# check_data(oil_df)
# print("=================================sub_df=================================")
# check_data(sub_df)
# print("=================================holiday_df=================================")
# check_data(holiday_df)
# print("=================================store_df=================================")
# check_data(store_df)
# print("=================================trans_df=================================")
# check_data(trans_df)

# 只有oil的dcoilwtico有缺值

In [ ]:
# 確認日期區間，以及將類別型欄位轉換為數值型態
# type_encoder = OrdinalEncoder()
# train_col = ['store_nbr', 'sales', 'onpromotion', 'family']
# train_df[train_col] = type_encoder.fit_transform(train_df[train_col])

# print('date range: ', min(train_df['date']), max(train_df['date']))
# print(train_df.head(20))

In [ ]:
# 調整連續型欄位的數值區間
# scaler = MinMaxScaler(feature_range=(0,1))

# for col in ['store_nbr', 'sales', 'onpromotion']:
#     scaled_data = scaler.fit_transform(train_df[col].values.reshape(-1,1))
#     train_df[col] = pd.Series(scaled_data.flatten())

# train_df.head(20)

In [ ]:
# # 計算每一天的平均sales、onpromotion
# train_df = train_df.groupby(['date']).agg({'sales':'mean', 'onpromotion':'mean'})
# train_df.head(20)

# x_train = train_df.copy()
# print(x_train.head())
# print(x_train.shape)

# y_train = train_df.sales.copy()
# y_train.head()

In [ ]:
print("=================================oil_df=================================")
check_data(oil_df)
oil_df.head(20)

In [ ]:
# fill missing date 填補中斷的日期，並以NAN插入
oil_df = oil_df.set_index("date").asfreq(freq = "D")


# fill the NaN value by interpolation 以插值處理NAN欄位
# both會抓前後的數值然後取平均
oil_df["dcoilwtico"] = oil_df["dcoilwtico"].interpolate(limit_direction="both")
oil_df.head(20)

In [ ]:
# 透過date欄位把oil資訊合併到train、test當中
train_df = train_df.merge(oil_df, on="date")
test_df = test_df.merge(oil_df, on="date")
train_df.head(5)

In [ ]:
# ref: https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/
# 我加入targetCol來指定我想要預測的欄位，而不是全部
def series_to_supervised(data, n_in=1, n_out=1, futureArr=None, targetCol=None, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in df.columns]
        
    # forecast sequence (t, t+1, ... t+n)    
    if futureArr != None:
        for i in range(0, n_out):
            for futureCol in futureArr:
                cols.append(df.shift(-i)[futureCol])
                if i == 0:
                    names += [('%s(t)' % (futureCol))]
                else:
                    names += [('%s(t+%d)' % (futureCol, i))]
    
    for i in range(0, n_out):
        if targetCol == None:
            cols.append(df.shift(-i))
            if i == 0:
                names += [('%s(t)' % (j)) for j in df.columns]
            else:
                names += [('%s(t+%d)' % (j, i)) for j in df.columns]
        else:
            cols.append(df.shift(-i)[targetCol])
            if i == 0:
                names += [('%s(t)' % (targetCol))]
            else:
                names += [('%s(t+%d)' % (targetCol, i))]
            
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
# 列出欄位中的唯一值，也就是store_nbr、family總共有幾種
store_nbr_types = train_df["store_nbr"].unique()
print(store_nbr_types)
family_types = train_df["family"].unique()
print(family_types)

test_date = test_df["date"].unique()
print(len(test_date), test_date)

In [ ]:
# ========== 以下是示範第一個iteration的結果 ==========
# 走訪所有商店/類型
for store_nbr_type in store_nbr_types:
    for family_type in family_types:
        # 符合特定商店、商品種類的訓練資料
        train_data = train_df[(train_df["store_nbr"] == store_nbr_type) & (train_df["family"] == family_type)]
        print(train_data.head(10))
        # 重置index並drop掉不需要的欄位
        train_data = train_data.reset_index()
        train_data = train_data.drop(columns = ["index", "date", "store_nbr", "family"])
        print(train_data.head(10))
        # 測試資料(用於填寫結果)        
        test_data = test_df[(test_df["store_nbr"] == store_nbr_type) & (test_df["family"] == family_type)]
        test_data = test_data.drop(columns = ["date", "store_nbr", "family"])
        break
    break

In [ ]:
# 把train和test合併，所以有些sales會是NAN
total_data = pd.concat([train_data, test_data])
total_data

In [ ]:
# Normalization 正規化，限縮至0~1
feature_name = total_data.columns

scaler = preprocessing.MinMaxScaler(feature_range = (0,1))

total_data = scaler.fit_transform(total_data)

total_data = pd.DataFrame(total_data, columns=feature_name)

total_data.head(20)

In [ ]:
# example
# 使用5天的資料，來預測後面2天的資料
futureArr = ["onpromotion", "dcoilwtico"]

res = series_to_supervised(total_data, 5, 2, futureArr=futureArr, targetCol="sales")
print(res.shape)
print(res.head(20))

In [ ]:
# 要用幾天來預測
past_days = 50
# 預測未來幾天
predict_days = 16
# 未來資料要使用於input的欄位
futureArr = ["onpromotion", "dcoilwtico"]
# 所要預測的欄位
targetCol = "sales"

train = series_to_supervised(total_data, past_days, predict_days, futureArr, targetCol)

split_ratio = 0.8

split_number = np.floor(len(train.index) * split_ratio)
split_number = np.int(split_number)

values = train.values


# split into train and validation sets
train = values[:split_number, :]
val = values[split_number:, :]

# split into input and outputs
train_x, train_y = train[:, :-predict_days], train[:, -predict_days:]
val_x, val_y = val[:, :-predict_days], val[:, -predict_days:]
# reshape input to be 3D [samples, timesteps, features]
train_x = train_x.reshape((train_x.shape[0], 1, train_x.shape[1]))
val_x = val_x.reshape((val_x.shape[0], 1, val_x.shape[1]))

# 預備用於預測的data(-17:-16代表使用倒數第17天的資料(2017/08/15))
prediction_data = series_to_supervised(total_data, past_days, predict_days, futureArr, targetCol, dropnan=False).values[-17:-16, :-predict_days]
prediction_data = prediction_data.reshape((prediction_data.shape[0], 1, prediction_data.shape[1]))

print(train_x.shape, train_y.shape, val_x.shape, val_y.shape, prediction_data.shape)
print(train.shape)
# print(train.head(20))

In [ ]:
# Model
model = keras.models.Sequential([
    keras.layers.LSTM(units=200, return_sequences=True, input_shape=(train_x.shape[1], train_x.shape[2])),
    keras.layers.LSTM(units=200, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(predict_days))
])

optimizer = keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mse"])

model.summary()

In [ ]:
early_stopping =  keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

model_result = model.fit(train_x, train_y, epochs=100, batch_size=32, validation_data=(val_x, val_y), verbose=2, shuffle=False, callbacks=[early_stopping])

In [ ]:
# plot history
plt.figure(figsize=(30, 10))

plt.subplot(1, 2, 1)
plt.plot(model_result.history["loss"], label="training")
plt.plot(model_result.history["val_loss"], label="validation")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(model_result.history["mse"], label="training")
plt.plot(model_result.history["val_mse"], label="validation")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

plt.show()

In [ ]:
prediction = model.predict(prediction_data)

prediction = np.squeeze(prediction) / scaler.scale_[0]

test_data["sales"] = prediction
test_data